# Train a model & track your experiments

In this notebook you will prepare features needed for training from a raw dataset and train an XGBoost model. The metrics and parameters associated with each training run will be tracked in a SageMaker Experiment.

In [ ]:
!pip install "sagemaker>=2.123.0"

In [ ]:
import sagemaker
import boto3
import numpy as np                                
import pandas as pd                               
import os                                         
from sagemaker import get_execution_role
from datetime import datetime

# Get default bucket
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/mlops-workshop'

# Get SageMaker Execution Role
role = get_execution_role()
region = boto3.Session().region_name

# SageMaker Session
sagemaker_session = sagemaker.session.Session()

### Retrieve variables from previous module

In [ ]:
%store -r

In [ ]:
print(train_uri)
print(test_uri)
print(val_uri)

## Training

To train a model in SageMaker, you create a training job. The training job includes the following information:

* The Amazon Elastic Container Registry path where the training code is stored.
* The URL of the Amazon Simple Storage Service (Amazon S3) bucket where you've stored the training data.
* The compute resources that you want SageMaker to use for model training. Compute resources are ML compute instances that are managed by SageMaker.
* The URL of the S3 bucket where you want to store the output of the job.

`xgboost` is an extremely popular, open-source package for gradient boosted trees.  It is computationally powerful, fully featured, and has been successfully used in many machine learning competitions.  Let's start with a simple `xgboost` model, trained using the Amazon SageMaker's XGBoost image.

In [ ]:
image_uri = sagemaker.image_uris.retrieve(region=region, framework='xgboost', version='latest')

## Track the training job in an Experiment

To ensure we are able to keep track of our parameters and metrics that correspond to the training job, we train a model as part of a Run that is associated with an Experiment

Experiments are organized as -
```
Experiment
    Run 1
    Run 2
    ...
```     
In this notebook, each time we run the Training job, it will correspond to a new Run in the Experiment. We can compare metrics across multiple runs to find the best set of parameters to train a model.

An estimator is a high level interface for SageMaker training. We will create an estimator object by supplying the required parameters, such as IAM role, compute instance count and type. and the S3 output path. 

We also supply hyperparameters for the algoirthm and then call its fit() method to start training the model.

### Track your training job in a Run

When using 1P algorithms, the metrics are tracked as part of the jobs listed in the Run. This can be found here. 

The run with th name `xgboost-` will have the job metrics

![](imgs/show-jobs.png)

![](imgs/view-job.png)

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.experiments import Run

model_path = f"s3://{bucket}/{prefix}/xgb_model"
experiment_name = f'mlops-workshop-experiment'

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path=model_path,
    role=role,
    sagemaker_session=sagemaker_session,
    keep_alive_period_in_seconds=3600
)

with Run(experiment_name = experiment_name, sagemaker_session = sagemaker_session) as run:      
    xgb_train.set_hyperparameters(
        objective="binary:logistic",
        num_round=50,
        max_depth=5,
        eta=0.2,
        gamma=4,
        min_child_weight=6,
        subsample=0.7,
        silent=0
    )

    xgb_train.fit(
        inputs = {
            "train": TrainingInput(
                s3_data=train_uri,
                content_type="text/csv"
            ),
            'validation': TrainingInput(
                s3_data=val_uri,
                content_type="text/csv"
            )
        }
    ) 

In [ ]:
trained_model_uri = xgb_train.model_data

In [ ]:
training_image = xgb_train.image_uri

In [ ]:
%store trained_model_uri
%store training_image

#### You can now move to the next section of the module `Track all models in a model registry`

The notebook used in that section is `sagemaker-register.ipynb`